# Лабораторная работа: Капрекар и неподвижные точки функции К(i)
* * * * * * * * * * * *
### Вычитание в соответствующей системе счисления
На вход поступает два массива list и основание системы счисления, происходит поразрядное вычитание

In [1]:
bad = ['A', 'B', 'C', 'D', 'E', 'F']

def substr(a, b, num_sys):
    res = []
    for i in range(len(b)):
        if (b[i] - a[i] >= 0): res.append(b[i] - a[i])
        else: 
            res.append(b[i] + num_sys - a[i])
            k = i + 1
            while ((b[k] == 0) and (k < len(b))):
                b[k] = b[k] + num_sys - 1
                k = k + 1
            #if (k == len(b):  - число < 0, в рамках наших вычислений такого не случится... можно добавить исключение
            b[k] = b[k] - 1
    ## для единственности найденных чисел сразу отсортируем его по убыванию
    res.reverse()
    for i in range(len(res)): 
        if(res[i] > 9): res[i] = bad[res[i] - 10]
    return ("".join(map(str,res)))

### Функция Капрекар. 
На вход поступает строка, в которой записано число, количество разрядов числа и система счисления  
На выходе функция возвращает строку - результат вычислений

In [2]:
def kaprekar(number, digit, num_sys):
    list_digits = [int(d, num_sys) for d in number]
    #while(digit > len(elem)):
    #    digits.append('0')
    list_digits.sort(reverse=True)
    max_sq = list_digits.copy()
    list_digits.sort()
    min_sq = list_digits.copy()
    #min_num = int("".join(map(str,min_sq))
    #max_num = int("".join(map(str,max_sq)))
    return substr(max_sq, min_sq, num_sys)

#### Введем переменные:  
list magicNumber содержит информацию о всех найденных неподвижных точках.  
            [ строка-запись числа - ключ, система счисления и длина до магического числа - пара значений по ключу]  
  
list usedNumber вспомогательный, чтобы найти цикл.

In [3]:
import math
magicNumber = {0: (math.inf, 2)}
usedNumber = []
Cycles = []

### Построение всех различных (для функции) чисел

Как будем строить число? Начнем с малого, заполним набор всех чисел для случая 2-разрядного числа с системе счисления по основанию 2. Это числа "11" и "10". Как далее происходит переход к большей разрядности или основанию С.С.?  

#### Разрядность:  
11 10       ->  (111, 110) + (100)  
111 110 100 -> (1111, 1110) + (1100) + (1000)   

#### Система счисления:  
11 10       + 22 21 20  
111 110 100 + 222 221 220 211 210 200


In [4]:
import numpy as np
batch_size = 10
# начальный массив возможных значений для d = 2, r = 2
start = ["11", "10"]

# строим таким образом, чтобы были только max значения (можно все min или премешанноые в виде 51423, 
# главное, чтобы одному набору цифр соответствовала только одна комбинация из них)

def add_d(arr, digit):
    tmp = []
    for i in arr:
        for k in range(3):
            if (int(i[digit - 2]) < k): break
            else: tmp.append(i + str(k))
    return tmp

def add_r(rez, number, digit, r):
    if (r > 10): number = number + bad[r - 11]
    else: number = number + (str(r - 1))
    for i in range (r):
        if (digit <= 1):
            rez.append(number)
            return rez
        else:
            add_r(rez, number, digit - 1, r - i)
    return rez

def in_Cycles(r):
    for arr in Cycles:
        if (arr[1] == r):
            for number in arr[0]:
                if (usedNumber[0] == number):
                    return True
    return False

### Process функция
На вход получает число, применяет к нему функцию капрекара, пока не найдется магическое число или процесс не зациклится

In [5]:
def process(number, d, r):
    usedNumber.clear()
    count = 0
    prev = "0"
    while True:
        prev = number
        usedNumber.append(number)
        number = kaprekar(number, d, r) 
        count = count + 1 
        if (int(number,r) == 0):
            break
        #else: 
            #print ("number is " + number)
        if (number == prev):
            if (magicNumber.get(number) == None):
                magicNumber[number] = (r, count)
                #print ("число " + number + " является неподвижной точкой: d = " + str(d) +" r = " + str(r) + "\n")
            elif (magicNumber[number][1] < count):
                magicNumber[number] = (r, count)
            break
        else:
            if(number in usedNumber): #найден цикл
                while (usedNumber[0] != number):
                    del usedNumber[0]
                if (not in_Cycles(r)): Cycles.append((usedNumber[:], r, count))
                break

In [6]:
for d in range(2,7):
    if (d > 2): start = add_d(start, d)
    for r in range(2,15):
        if (r > 2): 
            collection.extend(add_r([],"", d, r))
        else: 
            collection = start.copy()  
            
        for number in collection:
            process(number, d, r)

In [7]:
magicNumber

{0: (inf, 2),
 '01': (2, 2),
 '13': (5, 4),
 '25': (8, 3),
 '37': (11, 5),
 '49': (14, 3),
 '011': (2, 2),
 '132': (4, 4),
 '253': (6, 5),
 '374': (8, 6),
 '495': (10, 7),
 '5B6': (12, 8),
 '6D7': (14, 9),
 '1001': (2, 2),
 '0111': (2, 2),
 '3021': (4, 4),
 '3032': (5, 5),
 '6174': (10, 8),
 '10101': (2, 2),
 '01111': (2, 2),
 '20211': (3, 5),
 '41532': (6, 7),
 '62853': (9, 3),
 '83B74': (12, 13),
 '101101': (2, 2),
 '110001': (2, 2),
 '011111': (2, 2),
 '310221': (4, 7),
 '213312': (4, 2),
 '330201': (4, 3),
 '420432': (6, 6),
 '530421': (6, 4),
 '325523': (6, 2),
 '640632': (8, 17),
 '437734': (8, 2),
 '631764': (10, 5),
 '549945': (10, 2),
 '65BB56': (12, 2),
 '951A74': (13, 21),
 '76DD67': (14, 2)}

In [8]:
print(Cycles)

[(['03', '21'], 4, 3), (['05', '41', '23'], 6, 4), (['07', '61', '43'], 8, 4), (['53', '17'], 9, 5), (['09', '81', '63', '27', '45'], 10, 6), (['0B', 'A1', '83', '47', '29', '65'], 12, 7), (['93', '57', '1B'], 13, 6), (['0D', 'C1', 'A3', '67'], 14, 5), (['2B', '85'], 14, 3), (['022', '121'], 3, 3), (['242', '143'], 5, 5), (['363', '264'], 7, 6), (['484', '385'], 9, 7), (['5A5', '4A6'], 11, 8), (['6C6', '5C7'], 13, 9), (['1012', '1221'], 3, 3), (['2022', '1332'], 4, 5), (['3552', '3133', '1554', '4042', '4132', '3043'], 6, 9), (['5052', '5232', '3054'], 7, 6), (['6152', '5243', '3065'], 8, 6), (['6062', '6332', '3774', '4244', '1776'], 8, 8), (['7252', '5254', '3076'], 9, 6), (['7072', '7432', '5074'], 9, 6), (['9452', '7094', '9272', '7454', '3098'], 11, 8), (['9092', '9632', '7274', '5276', '5096'], 11, 8), (['7BB4', '7375', '4198', '8374', '5287', '6196'], 12, 10), (['3BB8', '8284', '6376'], 12, 7), (['B652', '90B4', 'B472', '9294', '7476', '30BA'], 13, 9), (['B0B2', 'B832', '9474', 

### Иллюстрация результатов анализа функции Капрекара

Финальный этап - наглядное демонстрирование результатов работы всей программы. Делать мы это будем с помощью таблицы, где строки отвечают количеству цифр в числе, а столбцы - системе счисления, в которой проделана операция вычитания.

In [9]:
import pandas as pd

result = np.zeros((5,13), dtype = list)

for key, val in magicNumber.items():
    if (val[0] == math.inf):
        continue
    row = len(key) - 2
    col = val[0] - 2
    if (result[row][col] == 0):
        result[row][col] = [1, key, val[1], 0, 0, 0]
    else:
        result[row][col][1] += " " + key;
        result[row][col][0] += 1
        if (result[row][col][2] < val[1]):
            result[row][col][2] = val[1]


In [10]:
for v in Cycles:
    row = len(v[0][0]) - 2
    col = v[1] - 2
    if (result[row][col] == 0):
        result[row][col] = [0," ", 0, 1, v[2], len(v[0])]
    elif (result[row][col][3] == 0):
        result[row][col][3] = 1
        result[row][col][4] = v[2]
        result[row][col][5] = len(v[0])
    else:
        result[row][col][3] += 1
        if (result[row][col][4] < v[2]):
            result[row][col][4] = v[2]
        if (result[row][col][5] < len(v[0])):
            result[row][col][5] = len(v[0])

In [11]:
table = pd.DataFrame(result, 
                     columns = [str(d) + " radix" for d in range(2, 15)],
                     index = [str(d) + "digit" for d in range(2,7)])
table

# 1 число - количество магических чисел
# 2 - строка содержащая магические числа, за исключением нуля (т.к. он явл магическим числом во всех случаях)
# 3 число - максимальное количество шагов, за которое будет достигнуто магическое число
# 4 число - количество циклов 
# 5 число - максимальное количество -\\- будет достигнут цикл
# 6 число - наибольшая длина цикла

,2 radix,3 radix,4 radix,5 radix,6 radix,7 radix,8 radix,9 radix,10 radix,11 radix,12 radix,13 radix,14 radix
2digit,"[1, 01, 2, 0, 0, 0]",0,"[0, , 0, 1, 3, 2]","[1, 13, 4, 0, 0, 0]","[0, , 0, 1, 4, 3]",0,"[1, 25, 3, 1, 4, 3]","[0, , 0, 1, 5, 2]","[0, , 0, 1, 6, 5]","[1, 37, 5, 0, 0, 0]","[0, , 0, 1, 7, 6]","[0, , 0, 1, 6, 3]","[1, 49, 3, 2, 5, 4]"
3digit,"[1, 011, 2, 0, 0, 0]","[0, , 0, 1, 3, 2]","[1, 132, 4, 0, 0, 0]","[0, , 0, 1, 5, 2]","[1, 253, 5, 0, 0, 0]","[0, , 0, 1, 6, 2]","[1, 374, 6, 0, 0, 0]","[0, , 0, 1, 7, 2]","[1, 495, 7, 0, 0, 0]","[0, , 0, 1, 8, 2]","[1, 5B6, 8, 0, 0, 0]","[0, , 0, 1, 9, 2]","[1, 6D7, 9, 0, 0, 0]"
4digit,"[2, 1001 0111, 2, 0, 0, 0]","[0, , 0, 1, 3, 2]","[1, 3021, 4, 1, 5, 2]","[1, 3032, 5, 0, 0, 0]","[0, , 0, 1, 9, 6]","[0, , 0, 1, 6, 3]","[0, , 0, 2, 8, 5]","[0, , 0, 2, 6, 3]","[1, 6174, 8, 0, 0, 0]","[0, , 0, 2, 8, 5]","[0, , 0, 2, 10, 6]","[0, , 0, 3, 9, 6]","[0, , 0, 1, 7, 3]"
5digit,"[2, 10101 01111, 2, 0, 0, 0]","[1, 20211, 5, 0, 0, 0]","[0, , 0, 1, 6, 2]","[0, , 0, 1, 6, 4]","[1, 41532, 7, 1, 4, 2]","[0, , 0, 1, 8, 5]","[0, , 0, 2, 8, 4]","[1, 62853, 3, 1, 9, 5]","[0, , 0, 3, 9, 4]","[0, , 0, 1, 10, 4]","[1, 83B74, 13, 1, 4, 2]","[0, , 0, 1, 10, 4]","[0, , 0, 2, 11, 4]"
6digit,"[3, 101101 110001 011111, 2, 0, 0, 0]","[0, , 0, 1, 4, 3]","[3, 310221 213312 330201, 7, 0, 0, 0]","[0, , 0, 1, 9, 5]","[3, 420432 530421 325523, 6, 1, 8, 3]","[0, , 0, 1, 10, 6]","[2, 640632 437734, 17, 1, 5, 3]","[0, , 0, 1, 22, 14]","[2, 631764 549945, 5, 1, 13, 7]","[0, , 0, 1, 15, 4]","[1, 65BB56, 2, 1, 18, 10]","[1, 951A74, 21, 0, 0, 0]","[1, 76DD67, 2, 3, 36, 12]"
